# BJP (network analysis)

In [ ]:
import pandas as pd

# Read the original CSV file
df = pd.read_csv('BJP.csv')

# Select the desired columns
selected_columns = df[['Date', 'User', 'Tweet','text']]

# Write the selected columns to a new CSV file
selected_columns.to_csv('BJPnetworkAnalysis.csv', index=False)


In [ ]:
import pandas as pd
import re
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
import networkx as nx
import matplotlib.pyplot as plt

# Load the CSV file
df = pd.read_csv('BJPnetworkAnalysis.csv')



# Tokenize tweets
df['tokenize_text'] = df['text'].apply(lambda x: word_tokenize(x))

# Remove stopwords from tweets
stop_words = set(stopwords.words('english'))
df['tokenize_text'] = df['tokenize_text'].apply(lambda x: [word for word in x if word.casefold() not in stop_words])

# Lemmatize words
lemmatizer = WordNetLemmatizer()
df['tokenize_text'] = df['tokenize_text'].apply(lambda x: [lemmatizer.lemmatize(word) for word in x])

# Create empty graph
G = nx.Graph()

# Add nodes for each user
for user in df['User'].unique():
    G.add_node(user)

# Add edges for retweets and mentions
for index, row in df.iterrows():
    user = row['User']
    mentions = re.findall(r'@(\w+)', row['tokenize_text'])
    for mention in mentions:
        if mention in G.nodes():
            G.add_edge(user, mention)
    if row['is_retweet']:
        original_user = row['retweet_from']
        if original_user in G.nodes():
            G.add_edge(user, original_user)

# Calculate centrality measures
betweenness = nx.betweenness_centrality(G)
closeness = nx.closeness_centrality(G)
degree = nx.degree_centrality(G)

# Print top 5 nodes for each centrality measure
print('Top 5 nodes by betweenness centrality:')
for node in sorted(betweenness, key=betweenness.get, reverse=True)[:5]:
    print(node, betweenness[node])
    
print('\nTop 5 nodes by closeness centrality:')
for node in sorted(closeness, key=closeness.get, reverse=True)[:5]:
    print(node, closeness[node])
    
print('\nTop 5 nodes by degree centrality:')
for node in sorted(degree, key=degree.get, reverse=True)[:5]:
    print(node, degree[node])

# Draw network graph
pos = nx.spring_layout(G, k=0.5, iterations=50)
nx.draw_networkx(G, pos, node_size=50, edge_color='#CCCCCC', with_labels=True)
plt.show()


In [1]:
# engagement rate

In [2]:
import csv
import snscrape.modules.twitter as sntwitter

# Open the CSV file containing the tweets
with open('tweets.csv', 'r', encoding='utf-8') as f:
    reader = csv.reader(f)
    next(reader)  # skip the header row
    
    # Iterate over the rows in the CSV file
    for row in reader:
        # Extract the tweet ID from the CSV file
        tweet_id = row[0]
        
        # Build the Twitter search query
        query = f"from:{row[1]} since_id:{tweet_id} -filter:retweets"
        
        # Get the tweet object using snscrape
        tweet = sntwitter.TwitterSearchScraper(query).get_items().__next__()
        
        # Extract the metrics from the tweet object
        retweets = tweet.replyCount
        likes = tweet.likeCount
        replies = tweet.replyCount
        
        # Append the data to a new row in the CSV file
        with open('tweets_with_metrics.csv', 'a', encoding='utf-8', newline='') as g:
            writer = csv.writer(g)
            writer.writerow(row + [retweets, likes, replies])


Error retrieving https://api.twitter.com/2/search/adaptive.json?include_profile_interstitial_type=1&include_blocking=1&include_blocked_by=1&include_followed_by=1&include_want_retweets=1&include_mute_edge=1&include_can_dm=1&include_can_media_tag=1&include_ext_has_nft_avatar=1&include_ext_is_blue_verified=1&include_ext_verified_type=1&skip_status=1&cards_platform=Web-12&include_cards=1&include_ext_alt_text=true&include_ext_limited_action_results=false&include_quote_count=true&include_reply_count=1&tweet_mode=extended&include_ext_collab_control=true&include_ext_views=true&include_entities=true&include_user_entities=true&include_ext_media_color=true&include_ext_media_availability=true&include_ext_sensitive_media_warning=true&include_ext_trusted_friends_metadata=true&send_error_codes=true&simple_quoted_tweet=true&q=from%3Anarendramodi+since_id%3A2014-05-12+15%3A01%3A56%2B00%3A00+-filter%3Aretweets&tweet_search_mode=live&count=20&query_source=spelling_expansion_revert_click&pc=1&spelling_cor

ScraperException: 4 requests to https://api.twitter.com/2/search/adaptive.json?include_profile_interstitial_type=1&include_blocking=1&include_blocked_by=1&include_followed_by=1&include_want_retweets=1&include_mute_edge=1&include_can_dm=1&include_can_media_tag=1&include_ext_has_nft_avatar=1&include_ext_is_blue_verified=1&include_ext_verified_type=1&skip_status=1&cards_platform=Web-12&include_cards=1&include_ext_alt_text=true&include_ext_limited_action_results=false&include_quote_count=true&include_reply_count=1&tweet_mode=extended&include_ext_collab_control=true&include_ext_views=true&include_entities=true&include_user_entities=true&include_ext_media_color=true&include_ext_media_availability=true&include_ext_sensitive_media_warning=true&include_ext_trusted_friends_metadata=true&send_error_codes=true&simple_quoted_tweet=true&q=from%3Anarendramodi+since_id%3A2014-05-12+15%3A01%3A56%2B00%3A00+-filter%3Aretweets&tweet_search_mode=live&count=20&query_source=spelling_expansion_revert_click&pc